In [ ]:
# You should run this after preprocessing
# Iterate the feature_extract function according to the "tracks_with_genre_small.csv"

In [2]:
import librosa as lb
import csv
import numpy as np
import pandas as pd
import os
import essentia.standard as es

In [4]:
AUDIO_FILE_HOME='../datasets/fma_medium_flatten/'
FEATURE_OUTPUT_HOME='./features'
META_FILE = '../preprocessing/datasets/tracks_with_genre_medium.csv'

In [6]:
def save_to_csv(data, columns, output_filename):
    df = pd.DataFrame([data], columns=columns)
    df.to_csv(f"{FEATURE_OUTPUT_HOME}/{output_filename}", mode='a', header=False, index=False)

def extract_feature_stats(y, sr, feature_func, *args, **kwargs):
    feature_values = feature_func(y=y, sr=sr, *args, **kwargs)
    mean = np.mean(feature_values, axis=1)
    var = np.var(feature_values, axis=1)
    return mean, var   # Returns mean and variance

def extract_zero_crossings(y, sr, filename, output_filename):
    zero_crossings = np.sum(lb.zero_crossings(y))
    save_to_csv([filename, zero_crossings], ['track_id', 'zero_crossings'], output_filename)

def extract_tempo(y, sr, filename, output_filename):
    onset_env = lb.onset.onset_strength(y=y, sr=sr)
    tempo, _ = lb.beat.beat_track(onset_envelope=onset_env, sr=sr)
    # Since tempo is a single value, variance is not applicable. We save tempo as is.
    save_to_csv([filename, tempo], ['track_id', 'tempo'], output_filename)

def extract_spectral_centroid(y, sr, filename, output_filename):
    spectral_centroid = lb.feature.spectral_centroid(y=y, sr=sr)
    sc_mean, sc_var = np.mean(spectral_centroid), np.var(spectral_centroid)
    save_to_csv([filename, sc_mean, sc_var], ['track_id', 'spectral_centroid_mean', 'spectral_centroid_var'], output_filename)

def extract_spectral_rolloff(y, sr, filename, output_filename):
    spectral_rolloff = lb.feature.spectral_rolloff(y=y, sr=sr)
    sr_mean, sr_var = np.mean(spectral_rolloff), np.var(spectral_rolloff)
    save_to_csv([filename, sr_mean, sr_var], ['track_id', 'spectral_rolloff_mean', 'spectral_rolloff_var'], output_filename)

def extract_chroma_stft(y, sr, filename, output_filename):
    chroma_mean, chroma_var = extract_feature_stats(y, sr, lb.feature.chroma_stft)
    data = [filename] + list(chroma_mean) + list(chroma_var)
    columns = ['track_id'] + [f'chroma_stft_{i}_mean' for i in range(12)] + [f'chroma_stft_{i}_var' for i in range(12)]
    save_to_csv(data, columns, output_filename)

def extract_mfccs(y, sr, filename, output_filename):
    mfccs_mean, mfccs_var = extract_feature_stats(y, sr, lb.feature.mfcc, n_mfcc=20)
    data = [filename] + list(mfccs_mean) + list(mfccs_var)
    columns = ['track_id'] + [f'MFCC_{i}_mean' for i in range(1, 21)] + [f'MFCC_{i}_var' for i in range(1, 21)]
    save_to_csv(data, columns, output_filename)

def extract_harmony_percussive(y, sr, filename, output_filename):
    y_harmonic, y_percussive = lb.effects.hpss(y)
    rms_harmonic_mean = np.mean(lb.feature.rms(y=y_harmonic))
    rms_harmonic_var = np.var(lb.feature.rms(y=y_harmonic))
    rms_percussive_mean = np.mean(lb.feature.rms(y=y_percussive))
    rms_percussive_var = np.var(lb.feature.rms(y=y_percussive))
    save_to_csv([filename, rms_harmonic_mean, rms_harmonic_var, rms_percussive_mean, rms_percussive_var], 
                ['track_id', 'rms_harmonic_mean', 'rms_harmonic_var', 'rms_percussive_mean', 'rms_percussive_var'], output_filename)
    

# Add more feature extraction functions if necessary, following the pattern shown above.

In [12]:
# Create CSV headers
headers = {
    #'zero_crossings_m.csv': ['track_id', 'zero_crossings'],
    #'tempo_m.csv': ['track_id', 'tempo'],
    #'spectral_centroid_m.csv': ['track_id', 'spectral_centroid_mean', 'spectral_centroid_var'],
    #'spectral_rolloff_m.csv': ['track_id', 'spectral_rolloff_mean', 'spectral_rolloff_var'],
    #'chroma_stft_m.csv': ['track_id'] + [f'chroma_stft_{i}_mean' for i in range(12)] + [f'chroma_stft_{i}_var' for i in range(12)],
    #'mfccs_m.csv': ['track_id'] + [f'MFCC_{i}_mean' for i in range(1, 21)] + [f'MFCC_{i}_var' for i in range(1, 21)],
    #'hpss_m.csv': ['track_id', 'rms_harmonic_mean', 'rms_harmonic_var', 'rms_percussive_mean', 'rms_percussive_var'],
    'key_scale_m.csv': ['track_id', 'key', 'scale', 'strength']
}

for key, value in headers.items():
    pd.DataFrame(columns=value).to_csv(f"{FEATURE_OUTPUT_HOME}/{key}", index=False)

In [5]:
def split_list_by_three(lst):
    # Determine the length of each chunk
    chunk_size = len(lst) // 3
    remainder = len(lst) % 3

    # Initialize the indices for slicing
    first_cut = chunk_size + (1 if remainder > 0 else 0)
    second_cut = first_cut + chunk_size + (1 if remainder > 1 else 0)

    # Split the list
    first_part = lst[0:first_cut]
    second_part = lst[first_cut:second_cut]
    third_part = lst[second_cut:]

    return first_part, second_part, third_part

In [8]:
file_list = sorted(os.listdir(AUDIO_FILE_HOME))
# first, second, third = split_list_by_three(file_list)

In [9]:
len(file_list)

25000

In [11]:
file_list_2 = file_list[3479:]

In [12]:
file_list_2

['020209.mp3',
 '020226.mp3',
 '020227.mp3',
 '020231.mp3',
 '020234.mp3',
 '020264.mp3',
 '020277.mp3',
 '020280.mp3',
 '020296.mp3',
 '020341.mp3',
 '020342.mp3',
 '020344.mp3',
 '020345.mp3',
 '020347.mp3',
 '020361.mp3',
 '020362.mp3',
 '020364.mp3',
 '020365.mp3',
 '020366.mp3',
 '020368.mp3',
 '020369.mp3',
 '020372.mp3',
 '020373.mp3',
 '020374.mp3',
 '020375.mp3',
 '020376.mp3',
 '020395.mp3',
 '020424.mp3',
 '020428.mp3',
 '020432.mp3',
 '020443.mp3',
 '020444.mp3',
 '020445.mp3',
 '020446.mp3',
 '020447.mp3',
 '020448.mp3',
 '020449.mp3',
 '020450.mp3',
 '020452.mp3',
 '020457.mp3',
 '020458.mp3',
 '020461.mp3',
 '020462.mp3',
 '020463.mp3',
 '020464.mp3',
 '020465.mp3',
 '020466.mp3',
 '020469.mp3',
 '020484.mp3',
 '020485.mp3',
 '020486.mp3',
 '020487.mp3',
 '020489.mp3',
 '020490.mp3',
 '020491.mp3',
 '020576.mp3',
 '020577.mp3',
 '020662.mp3',
 '020663.mp3',
 '020664.mp3',
 '020665.mp3',
 '020666.mp3',
 '020667.mp3',
 '020668.mp3',
 '020669.mp3',
 '020670.mp3',
 '020671.m

In [13]:
def extract_key_scale(filename, ouput_filename):  
    track_id = filename[:-4]
     
    loader = es.MonoLoader(filename=f"{AUDIO_FILE_HOME}{filename}")
    audio = loader()

    # Extract the key and scale (mode)
    key_extractor = es.KeyExtractor()
    key, scale, strength = key_extractor(audio)

    # print(f"Key: {key}, Scale: {scale}, Strength: {strength}")
    save_to_csv([track_id, key, scale, strength],['track_id', 'key', 'scale', 'strength'], ouput_filename)
    
for index, filename in enumerate(file_list):
    if index % 100 == 0:
        print(f'complete: {index} / ', filename)
    try:
        extract_key_scale(filename, 'key_scale_m.csv')
    except Exception as e:
        print(e)
    

complete: {index} /  000002.mp3
complete: {index} /  000698.mp3
complete: {index} /  001015.mp3
complete: {index} /  001366.mp3
Error while configuring MonoLoader: AudioLoader: Could not find stream information, error = End of file
complete: {index} /  001924.mp3
complete: {index} /  003571.mp3
complete: {index} /  003954.mp3
complete: {index} /  004233.mp3
complete: {index} /  004746.mp3
complete: {index} /  005156.mp3
Error while configuring MonoLoader: AudioLoader: Could not find stream information, error = End of file
complete: {index} /  006366.mp3
complete: {index} /  006719.mp3
complete: {index} /  007495.mp3
complete: {index} /  008370.mp3
complete: {index} /  009887.mp3
complete: {index} /  010250.mp3
complete: {index} /  010696.mp3
complete: {index} /  011321.mp3
complete: {index} /  011764.mp3
complete: {index} /  011933.mp3
complete: {index} /  012399.mp3
complete: {index} /  012691.mp3
complete: {index} /  013578.mp3
complete: {index} /  014069.mp3
complete: {index} /  014

[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input
[ WARNING  ] AudioLoader: invalid frame, skipping it: Invalid data found when processing input


Error while configuring MonoLoader: AudioLoader: Could not find stream information, error = End of file
complete: {index} /  098907.mp3
complete: {index} /  099028.mp3
Error while configuring MonoLoader: AudioLoader: Could not find stream information, error = End of file
complete: {index} /  099389.mp3
complete: {index} /  100547.mp3
complete: {index} /  102171.mp3
complete: {index} /  104271.mp3
complete: {index} /  105179.mp3
Error while configuring MonoLoader: AudioLoader: Could not find stream information, error = End of file
complete: {index} /  105907.mp3
complete: {index} /  106874.mp3
complete: {index} /  107381.mp3
complete: {index} /  107747.mp3
complete: {index} /  108222.mp3
complete: {index} /  108532.mp3
Error while configuring MonoLoader: AudioLoader: Could not find stream information, error = End of file
complete: {index} /  109050.mp3
complete: {index} /  109504.mp3
complete: {index} /  110159.mp3
complete: {index} /  110688.mp3
complete: {index} /  111224.mp3
complete

In [13]:

print("total number: ", len(file_list_2))
for index, filename in enumerate(file_list_2):
    track_id = filename[:-4]
    # print(f"track_id: ", track_id)
    # print(f"filename: ", filename)
    if index % 100 == 0:
        print('complete: {index} / ', filename)
    try:
        y, sr = lb.load(f"{AUDIO_FILE_HOME}{filename}") 
        extract_zero_crossings(y, sr, track_id, 'zero_crossings_m.csv')
        extract_tempo(y, sr, track_id, 'tempo_m.csv')
        extract_spectral_centroid(y, sr, track_id, 'spectral_centroid_m.csv')
        extract_spectral_rolloff(y, sr, track_id, 'spectral_rolloff_m.csv')
        extract_chroma_stft(y, sr, track_id, 'chroma_stft_m.csv')
        extract_mfccs(y, sr, track_id, 'mfccs_m.csv')  
        extract_harmony_percussive(y, sr, track_id, 'hpss_m.csv') 
    except Exception as e:
        print(index)
        print(e)
        

total number:  21521
complete: {index} /  020209.mp3
complete: {index} /  020749.mp3
complete: {index} /  021555.mp3


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


complete: {index} /  022088.mp3
complete: {index} /  022479.mp3
complete: {index} /  023482.mp3
complete: {index} /  024178.mp3
complete: {index} /  024831.mp3


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


complete: {index} /  025172.mp3
complete: {index} /  026244.mp3
complete: {index} /  026804.mp3
complete: {index} /  027551.mp3
complete: {index} /  028120.mp3
complete: {index} /  029041.mp3


...

18509

complete: {index} /  133624.mp3
complete: {index} /  134083.mp3
complete: {index} /  134934.mp3
complete: {index} /  135811.mp3
complete: {index} /  136882.mp3
complete: {index} /  137461.mp3
complete: {index} /  137907.mp3


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


complete: {index} /  138136.mp3
complete: {index} /  138918.mp3
complete: {index} /  139723.mp3
complete: {index} /  140332.mp3
complete: {index} /  140941.mp3
complete: {index} /  141593.mp3
complete: {index} /  142376.mp3
complete: {index} /  142671.mp3
complete: {index} /  143293.mp3


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...


20141

complete: {index} /  144472.mp3
complete: {index} /  145038.mp3
complete: {index} /  145689.mp3
complete: {index} /  145913.mp3
complete: {index} /  146942.mp3
complete: {index} /  147377.mp3
complete: {index} /  148082.mp3
complete: {index} /  148530.mp3
complete: {index} /  148987.mp3
complete: {index} /  149686.mp3
complete: {index} /  150395.mp3
complete: {index} /  151973.mp3
complete: {index} /  152721.mp3
complete: {index} /  155065.mp3


In [15]:
## merge features into a single file

file_names = ["tempo_m", "hpss_m", "spectral_centroid_m", "spectral_rolloff_m", "zero_crossings_m", "chroma_stft_m", "mfccs_m"]

# Using a list comprehension to read all dataframes into a list
dfs = [pd.read_csv(f"{FEATURE_OUTPUT_HOME}/{file_name}.csv") for file_name in file_names]

# Refactoring the merging process to be more concise
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on='track_id', how='outer')
    

In [16]:
merged_df.to_csv(f"{FEATURE_OUTPUT_HOME}/all_features_medium_with_var.csv")

In [11]:
NUM_FRAMES=200

In [12]:
# Extract raw datasets
# Create CSV headers
headers = {
    'spectral_centroid_raw.csv': ['track_id'] + [f'spectral_centroid_{i}' for i in range(NUM_FRAMES)],
    'spectral_rolloff_raw.csv': ['track_id'] + [f'spectral_rolloff_{i}' for i in range(NUM_FRAMES)],
    'chroma_stft_raw.csv': ['track_id'] + [f'chroma_stft_{i}_{j}' for i in range(1, 13) for j in range(NUM_FRAMES)],
    'mfccs_raw.csv': ['track_id'] + [f'MFCC_{i}_{j}' for i in range(1, 21) for j in range(NUM_FRAMES)],
    'hpss_raw.csv': ['track_id'] +
        [f'rms_harmonic_{i}' for i in range(NUM_FRAMES)] +
        [f'rms_percussive_{i}' for i in range(NUM_FRAMES)]
}

for key, value in headers.items():
    pd.DataFrame(columns=value).to_csv(f"{FEATURE_OUTPUT_HOME}/{key}", index=False)

In [13]:
# to extract original features. (without summation)
def extract_chroma_stft_raw(y, sr, filename, output_filename, num_frames=NUM_FRAMES):
    chroma_stft = extract_feature(y, sr, lb.feature.chroma_stft)
    # Limit the number of frames
    limited_chroma_stft = chroma_stft[:, :num_frames]
    data = [filename] + list(limited_chroma_stft.flatten())
    column_names = ['track_id'] + [f'chroma_stft_{i}_{j}' for i in range(1, 13) for j in range(num_frames)]
    save_to_csv(data, column_names, output_filename)
    
def extract_mfccs_raw(y, sr, filename, output_filename, n_mfcc=20, num_frames=NUM_FRAMES):
    mfccs = extract_feature(y, sr, lb.feature.mfcc, n_mfcc=n_mfcc)
    # Limit the number of frames
    limited_mfccs = mfccs[:, :num_frames]
    data = [filename] + list(limited_mfccs.flatten())
    column_names = ['track_id'] + [f'MFCC_{i}_{j}' for i in range(1, n_mfcc+1) for j in range(num_frames)]
    save_to_csv(data, column_names, output_filename)    
    
def extract_spectral_centroid_raw(y, sr, filename, output_filename, num_frames=NUM_FRAMES):
    spectral_centroid = extract_feature(y, sr, lb.feature.spectral_centroid)
    # Limit the number of frames
    limited_spectral_centroid = spectral_centroid[:, :num_frames]
    data = [filename] + list(limited_spectral_centroid.flatten())
    column_names = ['track_id'] + [f'spectral_centroid_{i}' for i in range(num_frames)]
    save_to_csv(data, column_names, output_filename)
    
def extract_spectral_rolloff_raw(y, sr, filename, output_filename, num_frames=NUM_FRAMES):
    spectral_rolloff = extract_feature(y, sr, lb.feature.spectral_rolloff)
    # Limit the number of frames
    limited_spectral_rolloff = spectral_rolloff[:, :num_frames]
    data = [filename] + list(limited_spectral_rolloff.flatten())
    column_names = ['track_id'] + [f'spectral_rolloff_{i}' for i in range(num_frames)]
    save_to_csv(data, column_names, output_filename)
    
def extract_harmony_percussive_raw(y, sr, filename, output_filename, num_frames=NUM_FRAMES):
    y_harmonic, y_percussive = lb.effects.hpss(y)
    
    # Extract RMS features for harmonic and percussive components
    rms_harmonic = lb.feature.rms(y=y_harmonic)
    rms_percussive = lb.feature.rms(y=y_percussive)
    
    # Limit the number of frames for each feature
    limited_rms_harmonic = rms_harmonic[:, :num_frames].flatten()
    limited_rms_percussive = rms_percussive[:, :num_frames].flatten()
    
    # Combine data and create column names
    data = [filename] + list(limited_rms_harmonic) + list(limited_rms_percussive)
    column_names = (
        ['track_id'] +
        [f'rms_harmonic_{i}' for i in range(num_frames)] +
        [f'rms_percussive_{i}' for i in range(num_frames)]
    )
    
    # Save to CSV
    save_to_csv(data, column_names, output_filename)

In [14]:
file_list = os.listdir(AUDIO_FILE_HOME)

for index, filename in enumerate(file_list):
    track_id = filename[:-4]
    # print(f"track_id: ", track_id)
    # print(f"filename: ", filename)
    if index % 100 == 0:
        print('complete: ', index)
    try:
        y, sr = lb.load(f"{AUDIO_FILE_HOME}{filename}") 

        extract_spectral_centroid_raw(y, sr, track_id, 'spectral_centroid_raw.csv')
        extract_spectral_rolloff_raw(y, sr, track_id, 'spectral_rolloff_raw.csv')
        extract_chroma_stft_raw(y, sr, track_id, 'chroma_stft_raw.csv')
        extract_mfccs_raw(y, sr, track_id, 'mfccs_raw.csv')  
        extract_harmony_percussive_raw(y, sr, track_id, 'hpss_raw.csv') 
    except Exception as e:
        print(index)
        print(e)

complete:  0
complete:  100
complete:  200
complete:  300
complete:  400
complete:  500
complete:  600
complete:  700


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!


complete:  800
complete:  900
complete:  1000
complete:  1100
complete:  1200
complete:  1300
complete:  1400
complete:  1500
complete:  1600
complete:  1700
complete:  1800


[src/libmpg123/parse.c:do_readahead():1099] warning: Cannot read next header, a one-frame stream? Duh...
/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)


1829

complete:  1900
complete:  2000


...

complete:  5300
complete:  5400
complete:  5500
complete:  5600
complete:  5700
complete:  5800
complete:  5900


[src/libmpg123/layer3.c:INT123_do_layer3():1801] error: dequantization failed!
/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  return f(*args, **kwargs)
Note: Illegal Audio-MPEG-Header 0x00000000 at offset 63168.
Note: Trying to resync...
Note: Skipped 1024 bytes in input.
[src/libmpg123/parse.c:wetwork():1365] error: Giving up resync after 1024 bytes - your stream is not nice... (maybe increasing resync limit could help).


5965
201 columns passed, passed data had 68 columns
complete:  6000
complete:  6100
complete:  6200
complete:  6300
complete:  6400
complete:  6500
complete:  6600
complete:  6700
complete:  6800
complete:  6900
complete:  7000
complete:  7100
complete:  7200
complete:  7300
complete:  7400
complete:  7500
complete:  7600


[src/libmpg123/layer3.c:INT123_do_layer3():1841] error: dequantization failed!


complete:  7700
complete:  7800
complete:  7900


In [15]:

file_names = ["hpss_raw", "spectral_centroid_raw", "spectral_rolloff_raw", "chroma_stft_raw", "mfccs_raw"]

# Using a list comprehension to read all dataframes into a list
dfs_raw = [pd.read_csv(f"{FEATURE_OUTPUT_HOME}/{file_name}.csv") for file_name in file_names]

# Refactoring the merging process to be more concise
raw_merged_df = dfs_raw[0]
for df in dfs_raw[1:]:
    raw_merged_df = raw_merged_df.merge(df, on='track_id', how='outer')

In [16]:
raw_merged_df.to_csv(f"{FEATURE_OUTPUT_HOME}/all_features_raw.csv")

In [17]:
raw_merged_df.shape

(7994, 7201)